In [1]:
import damask
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
cm=damask.ConfigMaterial()
cm=cm.load_DREAM3D('04.dream3d')
cm['phase']['Primary']=cm.load('Cu(2.1).yaml')
cm['homogenization']['direct']={'N_constituents': 1, 'mechanical': {'type': 'pass'}}
cm

phase:
  '1': t.b.d.
  Primary:
    lattice: cF
    mechanical:
      output: [F, F_e, F_p, O, P]
      elastic: {type: Hooke, C_11: 168600000000.0, C_12: 121500000000.0, C_44: 75590000000.0}
      plastic:
        type: phenopowerlaw
        output: [xi_sl, gamma_sl]
        N_sl: [12]
        n_sl: 20
        a_sl: 1.0
        h_0_sl-sl: 240000000.0
        xi_0_sl: [50000000.0]
        xi_inf_sl: [150000000.0]
        h_sl-sl: [1, 1, 1.4, 1.4, 1.4, 1.4, 1.4]
        dot_gamma_0_sl: 0.003

homogenization:
  direct:
    N_constituents: 1
    mechanical: {type: pass}

material:
  - constituents:
      - O: [0.3131131762355496, 0.7121126770766172, 0.5190264525444152, 0.3542135169267533]
        phase: 1
        v: 1.0
    homogenization: direct
  - constituents:
      - O: [0.22430078769369186, 0.8609957860909221, 0.2466231667936143, 0.38412553491580564]
        phase: 1
        v: 1.0
    homogenization: direct
  - constituents:
      - O: [0.22396536161370217, 0.8616980777950489, 0.24

In [3]:
cm.save()

In [4]:
grgeom= damask.Grid.load_DREAM3D('04.dream3d')

grgeom

cells:  122 × 84 × 1
size:   488.0 × 336.0 × 1.0 m³
origin: 0.0   0.0   0.0 m
# materials: 10125

In [5]:
grgeom.save('geom', compress=True)

In [6]:
def inversion(l,fill=0):
    return [inversion(i,fill) if isinstance(i,list) else\
            fill if i == 'x' else 'x' for i in l]


In [7]:
load_case = damask.Config(solver={'mechanical':'spectral_basic'},
                          loadstep=[])

In [8]:
dot_F = [[ 0, 0.001 , 0 ],
         [ 0, 0 , 0 ],
         [ 0, 0 , 0 ]]

loadstep = {'boundary_conditions':{'mechanical':{'dot_F':dot_F,
                                                 'P':inversion(dot_F)}},
                                   'discretization':{'t':200,'N':100},'f_out':5}
load_case['loadstep'].append(loadstep)


load_case

solver: {mechanical: spectral_basic}

loadstep:
  - boundary_conditions:
      mechanical:
        dot_F:
          - [0.001, 0, 0]
          - [0, x, 0]
          - [0, 0, x]
        P:
          - [x, x, x]
          - [x, 0, x]
          - [x, x, 0]
    discretization: {t: 200, N: 100}
    f_out: 5

In [9]:
load_case.save('tension-load.yaml')